In [1]:
import numpy as np
from scipy.optimize import minimize_scalar

In [2]:
# from parepy_toolbox.distributions import non_normal_approach_normal

# params_normal = {'loc': 0, 'scale': 1}
# result_normal = non_normal_approach_normal(1.3065, 'normal', params_normal)
# print(result_normal)

# params_gumbel_max = {'loc': 0.887487, 'scale': 5.1302}
# result_gumbel_max = non_normal_approach_normal(1.3065, 'gumbel max', params_gumbel_max)
# print(result_gumbel_max)

# params_log_normal = {'loc': 0.887487, 'scale': 5.1302}
# result_log_normal = non_normal_approach_normal(1.3065, 'log normal', params_log_normal)
# print(result_log_normal)

In [3]:
def obj(x, none_variable=None):
    simga_y = x[0]
    w = x[1]
    m = x[2]
    return simga_y * w - m

def grad_obj(x, none_variable=None):
    sigma_y = x[0]
    w = x[1]
    m = x[2]
    grad_g = [w, sigma_y, -1]
    return grad_g

def f_alpha(alpha,args):
    xk,d=args[0],args[1]
    xaux=xk+alpha*d
    f=obj(xaux)
    return f

# # Dataset
# f = {'type': 'normal', 'parameters': {'mean': 40, 'sigma': 5.}, 'stochastic variable': False}
# p = {'type': 'normal', 'parameters': {'mean': 50, 'sigma': 2.5}, 'stochastic variable': False}
# w = {'type': 'normal', 'parameters': {'mean': 1000, 'sigma': 200}, 'stochastic variable': False}
# var = [f, p, w]

# # PAREpy setup
# setup = {
#                 'tolerance': 1e-6, 
#                 'max iterations': 20,
#                 'numerical model': 'fosm', 
#                 'variables settings': var, 
#                 'number of state limit functions or constraints': 1, 
#                 'none variable': None,
#                 'objective function': obj,
#                 'gradient objective function': grad_obj,
#                 'name simulation': None,
#         }

def deterministic_algorithm_structural_analysis(setup: dict) -> tuple[float, int]:
    try:
        # Verifica se o setup é um dicionário
        if not isinstance(setup, dict):
            raise TypeError('The setup parameter must be a dictionary.')
        
        # Verifica se as chaves obrigatórias estão presentes
        required_keys = [
            'tolerance', 'max iterations', 'numerical model', 'variables settings',
            'number of state limit functions or constraints', 'none variable',
            'objective function', 'gradient objective function', 'name simulation'
        ]
        for key in required_keys:
            if key not in setup:
                raise ValueError(f'The setup parameter must have the key: {key}.')
        
        # Extrai as variáveis e constrói o dicionário no formato especificado
        variables = setup['variables settings']
        params_adapt  = {}  # Dicionário para armazenar as variáveis convertidas
        
        for i, var in enumerate(variables):
            if not isinstance(var, dict):
                raise TypeError('Each variable in "variables settings" must be a dictionary.')
            
            if 'parameters' not in var or not isinstance(var['parameters'], dict):
                raise ValueError('Each variable must have a "parameters" key with a dictionary value.')
            
            if 'mean' not in var['parameters'] or 'sigma' not in var['parameters']:
                raise ValueError('Each variable must have "mean" and "sigma" in its parameters.')
            
            mean = var['parameters']['mean']
            std = var['parameters']['sigma']
            
            if var['type'] == 'normal':
                params_adapt[f'var{i}'] = {
                    'type': 'normal',
                    'params': {
                        'mu': mean,
                        'sigma': std
                    }
                }
            elif var['type'] == 'lognormal':
                epsilon = np.sqrt(np.log(1 + (std / mean) ** 2))
                lambdaa = np.log(mean) - 0.5 * epsilon ** 2
                params_adapt[f'var{i}'] = {
                    'type': 'lognormal',
                    'params': {
                        'lambda': lambdaa,
                        'epsilon': epsilon
                    }
                }
            elif var['type'] == 'gumbel max':
                gamma = 0.577215665  
                beta = np.pi / (np.sqrt(6) * std)
                calpha = mean - gamma / beta
                params_adapt[f'var{i}'] = {
                    'type': 'gumbel max',
                    'params': {
                        'alpha': calpha,
                        'beta': beta
                    }
                }
            elif var['type'] == 'gumbel min':
                gamma = 0.577215665 
                beta = np.pi / (np.sqrt(6) * std)
                alpha = mean + gamma / beta
                params_adapt[f'var{i}'] = {
                    'type': 'gumbel min',
                    'params': {
                        'alpha': alpha,
                        'beta': beta
                    }
                }
                 
        tol = setup['tolerance']
        max_iter = setup['max iterations']
        none_variable = setup['none variable']
        obj = setup['objective function']
        grad_obj = setup['gradient objective function']

        for index, value in params_adapt.items():
            print(f"index: {index}, \nvalue: {value}")

        print(f"mu: {mean}, \nsigma: {std}, \ntol: {tol}, \nmax_iter: {max_iter}, \nnone_variable: {none_variable}")
        
        # Fixed in this algorithm
        beta_list = [10000]
        error = 1000
        iter = 0
        step = 1

        x = np.transpose(np.array([mu.copy()]))
        mu = x.copy()
        jacobian_xy = np.diag(std)
        jacobian_xy_trans = np.transpose(jacobian_xy)
        jacobian_yx = np.linalg.inv(jacobian_xy)
        y = jacobian_yx @ (x - mu)
        x = jacobian_xy @ y + mu

        while (error > tol and iter < max_iter):
            beta = np.linalg.norm(y)
            beta_list.append(beta)
            g_y = obj(x.flatten().tolist())
            grad_g_x = grad_obj(x.flatten().tolist())
            grad_g_y = np.dot(jacobian_xy_trans, np.transpose(np.array([grad_g_x])))
            num = (np.transpose(grad_g_y) @ y - g_y)
            norm = np.linalg.norm(grad_g_y)
            norm2 = norm ** 2
            #alpha = grad_g_y / norm
            #aux = g_y / norm
            #y = -alpha * (beta + aux)
            d = grad_g_y @ (num / norm2) - y
            #step = minimize_scalar(f_alpha, bounds=(.001, 1), args=([y, d]), method='bounded')
            #print(step.x)
            #y += step.x * d
            y += step * d
            error = np.abs(beta_list[iter + 1] - beta_list[iter])
            x = jacobian_xy @ y + mu
            iter += 1
        return float(beta), iter

    except TypeError as e:
        print(f"TypeError: {e}")
        return None, iter

    except Exception as e:
            print(f"Exception: {e}")
            return None, iter

In [4]:
# Dataset
f = {'type': 'lognormal', 'parameters': {'mean': 40, 'sigma': 5.}, 'stochastic variable': False}
p = {'type': 'normal', 'parameters': {'mean': 50, 'sigma': 2.5}, 'stochastic variable': False}
w = {'type': 'normal', 'parameters': {'mean': 1000, 'sigma': 200}, 'stochastic variable': False}
var = [f, p, w]

# PAREpy setup
setup = {
                'tolerance': 1e-6, 
                'max iterations': 20,
                'numerical model': 'fosm', 
                'variables settings': var, 
                'number of state limit functions or constraints': 1, 
                'none variable': None,
                'objective function': obj,
                'gradient objective function': grad_obj,
                'name simulation': None,
        }

# Call algorithm
beta = deterministic_algorithm_structural_analysis(setup)
print(beta)

index: var0, 
value: {'type': 'lognormal', 'params': {'lambda': np.float64(3.6811273608459536), 'epsilon': np.float64(0.1245158083777528)}}
index: var1, 
value: {'type': 'normal', 'params': {'mu': 50, 'sigma': 2.5}}
index: var2, 
value: {'type': 'normal', 'params': {'mu': 1000, 'sigma': 200}}
mu: 1000, 
sigma: 200, 
tol: 1e-06, 
max_iter: 20, 
none_variable: None
Exception: cannot access local variable 'mu' where it is not associated with a value
(None, 0)
